# Reinforcement Learning - Clase 2: Entrenamiento con Stable-Baselines3

## Continuación de reinforcement_learning_clase.ipynb

En el notebook anterior aprendimos:
- Conceptos fundamentales de RL (agente, entorno, recompensa, política)
- Algoritmos tabulares (SARSA, Q-Learning)
- Deep Q-Network (DQN) desde cero
- Gymnasium para entornos

En este notebook aprenderemos:
- **Stable-Baselines3**: La librería estándar para entrenar agentes RL
- **Algoritmos avanzados**: PPO, A2C, SAC, TD3
- **Entrenamiento práctico**: Callbacks, logging, guardado de modelos
- **Proyectos reales**: Conexión con `04_proyectos_avanzados/`

---

## Índice

1. [¿Por qué Stable-Baselines3?](#1-porque-sb3)
2. [Instalación y Setup](#2-instalacion)
3. [Tu Primer Agente con SB3](#3-primer-agente)
4. [Algoritmos Disponibles](#4-algoritmos)
5. [Políticas (Redes Neuronales)](#5-politicas)
6. [Entrenamiento Avanzado](#6-entrenamiento)
7. [Callbacks y Logging](#7-callbacks)
8. [Guardar y Cargar Modelos](#8-guardar-cargar)
9. [Evaluación de Agentes](#9-evaluacion)
10. [Hiperparámetros](#10-hiperparametros)
11. [Proyectos Prácticos](#11-proyectos)

---
<a id='1-porque-sb3'></a>
# 1. ¿Por qué Stable-Baselines3?

## El Problema: Implementar RL es Difícil

Implementar algoritmos de RL correctamente es sorprendentemente difícil:

| Desafío | Descripción |
|---------|-------------|
| **Bugs sutiles** | Un signo mal puesto puede hacer que el agente no aprenda |
| **Hiperparámetros** | Cada algoritmo tiene 10+ parámetros que ajustar |
| **Reproducibilidad** | Pequeños cambios causan grandes diferencias |
| **Eficiencia** | Optimizaciones de bajo nivel para velocidad |
| **Debugging** | Es difícil saber si el código está mal o el algoritmo es lento |

## La Solución: Stable-Baselines3

**Stable-Baselines3 (SB3)** es una librería que proporciona implementaciones fiables y probadas de los algoritmos de RL más importantes.

### Características

| Característica | Beneficio |
|----------------|----------|
| **Implementaciones correctas** | Probadas en benchmarks, sin bugs |
| **API unificada** | Todos los algoritmos tienen la misma interfaz |
| **Integración con Gymnasium** | Funciona directamente con cualquier entorno |
| **PyTorch backend** | Fácil de extender y modificar |
| **Documentación excelente** | Tutoriales, ejemplos, API reference |
| **Comunidad activa** | Mantenido y actualizado regularmente |

### Cuándo Usar SB3 vs Implementación Propia

| Usar SB3 | Implementar tú mismo |
|----------|---------------------|
| Proyectos reales | Aprender cómo funcionan los algoritmos |
| Necesitas resultados rápidos | Investigación de nuevos algoritmos |
| Comparar algoritmos | Modificaciones profundas al algoritmo |
| Producción | Propósitos educativos |

---
<a id='2-instalacion'></a>
# 2. Instalación y Setup

In [ ]:
# INSTALACIÓN
# ===========
# Ejecutar solo si no están instaladas las dependencias

# Stable-Baselines3 (incluye PyTorch si no lo tienes)
# !pip install stable-baselines3[extra]

# Para entornos Box2D (LunarLander, BipedalWalker)
# !pip install gymnasium[box2d]

# Para visualización de entrenamiento
# !pip install tensorboard

In [ ]:
# IMPORTS
# =======

# Stable-Baselines3
try:
    from stable_baselines3 import PPO, A2C, DQN, SAC, TD3
    from stable_baselines3.common.env_util import make_vec_env
    from stable_baselines3.common.evaluation import evaluate_policy
    from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback
    from stable_baselines3.common.monitor import Monitor
    SB3_AVAILABLE = True
    print("✅ Stable-Baselines3 disponible")
except ImportError:
    SB3_AVAILABLE = False
    print("❌ Stable-Baselines3 no instalado. Ejecuta: pip install stable-baselines3[extra]")

# Gymnasium
import gymnasium as gym
print("✅ Gymnasium disponible")

# Utilidades
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os

# Configuración para Windows
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

print("\n✅ Imports completados")

---
<a id='3-primer-agente'></a>
# 3. Tu Primer Agente con SB3

Entrenar un agente con SB3 requiere solo **3 líneas de código**:

In [ ]:
# TU PRIMER AGENTE EN 3 LÍNEAS
# ============================

if SB3_AVAILABLE:
    # 1. CREAR EL MODELO
    # - PPO: Algoritmo (Proximal Policy Optimization)
    # - "MlpPolicy": Red neuronal fully-connected (Multi-Layer Perceptron)
    # - "CartPole-v1": Entorno de Gymnasium
    model = PPO("MlpPolicy", "CartPole-v1", verbose=1)

    # 2. ENTRENAR
    # - total_timesteps: Número total de pasos de interacción
    # - CartPole se resuelve en ~20,000 pasos con PPO
    model.learn(total_timesteps=20_000)

    # 3. EVALUAR
    # - Crea un entorno nuevo para evaluar
    # - Ejecuta 10 episodios y calcula estadísticas
    env = gym.make("CartPole-v1")
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
    print(f"\n🎯 Recompensa media: {mean_reward:.2f} ± {std_reward:.2f}")
    print(f"   (Máximo posible en CartPole: 500)")
    env.close()
else:
    print("⚠️ Instala stable-baselines3 para ejecutar este código")

In [ ]:
# VER EL AGENTE EN ACCIÓN
# =======================

if SB3_AVAILABLE:
    # Crear entorno con renderizado
    env = gym.make("CartPole-v1", render_mode="human")
    
    # Ejecutar un episodio
    obs, info = env.reset()
    total_reward = 0
    
    for step in range(500):
        # El modelo predice la acción
        action, _ = model.predict(obs, deterministic=True)
        
        # Ejecutar acción
        obs, reward, terminated, truncated, info = env.step(action)
        total_reward += reward
        
        if terminated or truncated:
            break
    
    print(f"\n🎮 Episodio completado: {total_reward} pasos")
    env.close()

---
<a id='4-algoritmos'></a>
# 4. Algoritmos Disponibles

SB3 incluye los algoritmos más importantes de RL moderno:

## Tabla Resumen

| Algoritmo | Tipo | Acciones | Uso Principal |
|-----------|------|----------|---------------|
| **PPO** | On-policy, Actor-Critic | Discretas y Continuas | General, muy estable |
| **A2C** | On-policy, Actor-Critic | Discretas y Continuas | Más simple que PPO |
| **DQN** | Off-policy, Value-based | Solo Discretas | Juegos, estados discretos |
| **SAC** | Off-policy, Actor-Critic | Solo Continuas | Robótica, control |
| **TD3** | Off-policy, Actor-Critic | Solo Continuas | Robótica, alternativa a SAC |
| **DDPG** | Off-policy, Actor-Critic | Solo Continuas | Predecesor de TD3/SAC |
| **HER** | Extensión | Con los anteriores | Tareas con metas |

## ¿Cuál Elegir?

```
¿Tu entorno tiene acciones DISCRETAS o CONTINUAS?
                    |
        +-----------+-----------+
        |                       |
    DISCRETAS                CONTINUAS
        |                       |
  ¿Necesitas                ¿Necesitas
  sample efficiency?        sample efficiency?
        |                       |
    +---+---+               +---+---+
    |       |               |       |
   Sí      No              Sí      No
    |       |               |       |
   DQN    PPO             SAC     PPO
                          TD3
```

### Recomendaciones Prácticas

| Situación | Algoritmo | Razón |
|-----------|-----------|-------|
| **No sabes cuál usar** | PPO | Funciona bien casi siempre |
| **Videojuegos (Atari)** | DQN | Diseñado para esto |
| **Robótica / Control** | SAC o TD3 | Eficientes con acciones continuas |
| **Muchos entornos paralelos** | PPO o A2C | Aprovechan bien la paralelización |
| **Poca experiencia disponible** | SAC o DQN | Reutilizan experiencias (off-policy) |

In [ ]:
# COMPARACIÓN DE ALGORITMOS
# =========================
# Entrenamos varios algoritmos en el mismo entorno y comparamos

if SB3_AVAILABLE:
    from stable_baselines3 import PPO, A2C, DQN
    
    # Entorno simple para comparar
    env_id = "CartPole-v1"
    timesteps = 10_000  # Pocos pasos para demo rápida
    
    resultados = {}
    
    # PPO
    print("Entrenando PPO...")
    model_ppo = PPO("MlpPolicy", env_id, verbose=0)
    model_ppo.learn(total_timesteps=timesteps)
    mean_r, _ = evaluate_policy(model_ppo, gym.make(env_id), n_eval_episodes=10, warn=False)
    resultados["PPO"] = mean_r
    print(f"  PPO: {mean_r:.1f}")
    
    # A2C
    print("Entrenando A2C...")
    model_a2c = A2C("MlpPolicy", env_id, verbose=0)
    model_a2c.learn(total_timesteps=timesteps)
    mean_r, _ = evaluate_policy(model_a2c, gym.make(env_id), n_eval_episodes=10, warn=False)
    resultados["A2C"] = mean_r
    print(f"  A2C: {mean_r:.1f}")
    
    # DQN
    print("Entrenando DQN...")
    model_dqn = DQN("MlpPolicy", env_id, verbose=0)
    model_dqn.learn(total_timesteps=timesteps)
    mean_r, _ = evaluate_policy(model_dqn, gym.make(env_id), n_eval_episodes=10, warn=False)
    resultados["DQN"] = mean_r
    print(f"  DQN: {mean_r:.1f}")
    
    # Visualizar
    plt.figure(figsize=(8, 4))
    plt.bar(resultados.keys(), resultados.values(), color=['blue', 'green', 'orange'])
    plt.ylabel("Recompensa Media")
    plt.title(f"Comparación de Algoritmos en {env_id} ({timesteps:,} pasos)")
    plt.axhline(y=500, color='red', linestyle='--', label='Máximo')
    plt.legend()
    plt.show()

---
<a id='5-politicas'></a>
# 5. Políticas (Redes Neuronales)

La "política" en SB3 es la red neuronal que decide las acciones.

## Políticas Predefinidas

| Política | Descripción | Uso |
|----------|-------------|-----|
| `MlpPolicy` | Red fully-connected | Estados vectoriales (ej: CartPole) |
| `CnnPolicy` | Red convolucional | Estados imagen (ej: Atari) |
| `MultiInputPolicy` | Combina MLP + CNN | Estados mixtos (Dict) |

## Arquitectura por Defecto de MlpPolicy

```
Observación (estado)
        ↓
┌───────────────────┐
│ Linear(obs_dim, 64)│ ← Capa 1
│     + Tanh        │
└───────────────────┘
        ↓
┌───────────────────┐
│ Linear(64, 64)    │ ← Capa 2
│     + Tanh        │
└───────────────────┘
        ↓
┌───────────────────┐
│ Linear(64, n_act) │ ← Capa de salida
└───────────────────┘
        ↓
Acción (o distribución de acciones)
```

In [ ]:
# PERSONALIZAR LA ARQUITECTURA
# ============================

if SB3_AVAILABLE:
    # Definir arquitectura custom
    # - net_arch: Lista de capas para policy y value networks
    # - [dict(pi=[...], vf=[...])] separa las arquitecturas
    
    policy_kwargs = dict(
        net_arch=[128, 128, 64]  # 3 capas: 128, 128, 64 neuronas
    )
    
    # Crear modelo con arquitectura custom
    model = PPO(
        "MlpPolicy",
        "CartPole-v1",
        policy_kwargs=policy_kwargs,
        verbose=1
    )
    
    print("Arquitectura de la política:")
    print(model.policy)

In [ ]:
# ARQUITECTURAS SEPARADAS PARA ACTOR Y CRITIC
# ============================================

if SB3_AVAILABLE:
    # En algoritmos Actor-Critic (PPO, A2C), puedes separar las arquitecturas
    
    policy_kwargs = dict(
        net_arch=dict(
            pi=[64, 64],    # Actor (política): 2 capas de 64
            vf=[128, 128]   # Critic (valor): 2 capas de 128
        )
    )
    
    model = PPO(
        "MlpPolicy",
        "CartPole-v1",
        policy_kwargs=policy_kwargs,
        verbose=0
    )
    
    print("✅ Modelo con arquitecturas separadas creado")
    print(f"   Actor: 2 capas de 64")
    print(f"   Critic: 2 capas de 128")

---
<a id='6-entrenamiento'></a>
# 6. Entrenamiento Avanzado

## Entornos Vectorizados

Para entrenar más rápido, usa múltiples entornos en paralelo:

In [ ]:
# ENTRENAMIENTO CON ENTORNOS PARALELOS
# ====================================

if SB3_AVAILABLE:
    from stable_baselines3.common.env_util import make_vec_env
    
    # Crear 4 entornos paralelos
    # - Cada entorno recolecta experiencias simultáneamente
    # - El entrenamiento es ~4x más rápido
    vec_env = make_vec_env("CartPole-v1", n_envs=4)
    
    # PPO funciona especialmente bien con múltiples entornos
    model = PPO(
        "MlpPolicy",
        vec_env,
        verbose=1,
        n_steps=1024,      # Pasos por entorno antes de actualizar
        batch_size=64,     # Tamaño del minibatch
        n_epochs=10,       # Épocas de optimización por actualización
    )
    
    print(f"\n🚀 Entrenando con {vec_env.num_envs} entornos paralelos...")
    model.learn(total_timesteps=20_000)
    
    # Evaluar
    mean_r, std_r = evaluate_policy(model, gym.make("CartPole-v1"), n_eval_episodes=10)
    print(f"\n🎯 Resultado: {mean_r:.1f} ± {std_r:.1f}")

---
<a id='7-callbacks'></a>
# 7. Callbacks y Logging

Los callbacks permiten ejecutar código durante el entrenamiento:
- Guardar checkpoints
- Evaluar periódicamente
- Early stopping
- Logging personalizado

In [ ]:
# CALLBACKS BÁSICOS
# =================

if SB3_AVAILABLE:
    from stable_baselines3.common.callbacks import (
        EvalCallback,
        CheckpointCallback,
        CallbackList
    )
    
    # Directorio para guardar
    log_dir = "./logs/"
    os.makedirs(log_dir, exist_ok=True)
    
    # 1. CheckpointCallback: Guarda el modelo cada N pasos
    checkpoint_callback = CheckpointCallback(
        save_freq=5000,              # Guardar cada 5000 pasos
        save_path=log_dir,           # Directorio
        name_prefix="ppo_cartpole"   # Prefijo del archivo
    )
    
    # 2. EvalCallback: Evalúa y guarda el mejor modelo
    eval_env = gym.make("CartPole-v1")
    eval_callback = EvalCallback(
        eval_env,
        best_model_save_path=log_dir,   # Guarda el mejor
        log_path=log_dir,               # Logs de evaluación
        eval_freq=2000,                 # Evaluar cada 2000 pasos
        n_eval_episodes=5,              # 5 episodios por evaluación
        deterministic=True
    )
    
    # Combinar callbacks
    callbacks = CallbackList([checkpoint_callback, eval_callback])
    
    # Entrenar con callbacks
    model = PPO("MlpPolicy", "CartPole-v1", verbose=1)
    model.learn(total_timesteps=20_000, callback=callbacks)
    
    print(f"\n✅ Modelos guardados en {log_dir}")
    print(f"   Archivos: {os.listdir(log_dir)}")

In [ ]:
# TENSORBOARD PARA VISUALIZACIÓN
# ==============================

if SB3_AVAILABLE:
    # SB3 integra automáticamente con TensorBoard
    
    model = PPO(
        "MlpPolicy",
        "CartPole-v1",
        verbose=1,
        tensorboard_log="./tensorboard_logs/"  # Directorio de logs
    )
    
    # Entrenar (los logs se guardan automáticamente)
    model.learn(total_timesteps=10_000)
    
    print("\n📊 Para visualizar en TensorBoard:")
    print("   tensorboard --logdir=./tensorboard_logs/")
    print("   Luego abre http://localhost:6006 en tu navegador")

---
<a id='8-guardar-cargar'></a>
# 8. Guardar y Cargar Modelos

In [ ]:
# GUARDAR Y CARGAR MODELOS
# ========================

if SB3_AVAILABLE:
    # Entrenar un modelo
    model = PPO("MlpPolicy", "CartPole-v1", verbose=0)
    model.learn(total_timesteps=10_000)
    
    # GUARDAR
    # -------
    model.save("ppo_cartpole")  # Guarda como ppo_cartpole.zip
    print("✅ Modelo guardado: ppo_cartpole.zip")
    
    # Eliminar el modelo de memoria
    del model
    
    # CARGAR
    # ------
    model = PPO.load("ppo_cartpole")  # Carga desde ppo_cartpole.zip
    print("✅ Modelo cargado")
    
    # Verificar que funciona
    env = gym.make("CartPole-v1")
    mean_r, _ = evaluate_policy(model, env, n_eval_episodes=5)
    print(f"🎯 Recompensa tras cargar: {mean_r:.1f}")
    
    # CONTINUAR ENTRENAMIENTO
    # -----------------------
    # Necesitas pasar el entorno al cargar
    model = PPO.load("ppo_cartpole", env=gym.make("CartPole-v1"))
    model.learn(total_timesteps=5_000)  # Continúa entrenando
    print("✅ Entrenamiento continuado")

---
<a id='9-evaluacion'></a>
# 9. Evaluación de Agentes

In [ ]:
# EVALUACIÓN COMPLETA
# ===================

if SB3_AVAILABLE:
    from stable_baselines3.common.evaluation import evaluate_policy
    
    # Entrenar modelo
    model = PPO("MlpPolicy", "CartPole-v1", verbose=0)
    model.learn(total_timesteps=20_000)
    
    # Crear entorno de evaluación
    eval_env = gym.make("CartPole-v1")
    
    # Evaluar con estadísticas
    mean_reward, std_reward = evaluate_policy(
        model,
        eval_env,
        n_eval_episodes=20,       # Número de episodios
        deterministic=True,        # Acciones deterministas (sin exploración)
        return_episode_rewards=False
    )
    
    print(f"📊 Evaluación (20 episodios):")
    print(f"   Recompensa media: {mean_reward:.2f}")
    print(f"   Desviación estándar: {std_reward:.2f}")
    print(f"   Rango esperado: [{mean_reward-2*std_reward:.1f}, {mean_reward+2*std_reward:.1f}]")

In [ ]:
# EVALUACIÓN DETALLADA CON HISTORIAL
# ===================================

if SB3_AVAILABLE:
    # Obtener recompensas individuales
    episode_rewards, episode_lengths = evaluate_policy(
        model,
        eval_env,
        n_eval_episodes=20,
        return_episode_rewards=True
    )
    
    # Visualizar distribución
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    axes[0].hist(episode_rewards, bins=10, edgecolor='black')
    axes[0].axvline(np.mean(episode_rewards), color='red', linestyle='--', label=f'Media: {np.mean(episode_rewards):.1f}')
    axes[0].set_xlabel("Recompensa por Episodio")
    axes[0].set_ylabel("Frecuencia")
    axes[0].set_title("Distribución de Recompensas")
    axes[0].legend()
    
    axes[1].plot(episode_rewards, 'o-')
    axes[1].axhline(np.mean(episode_rewards), color='red', linestyle='--')
    axes[1].set_xlabel("Episodio")
    axes[1].set_ylabel("Recompensa")
    axes[1].set_title("Recompensa por Episodio")
    
    plt.tight_layout()
    plt.show()

---
<a id='10-hiperparametros'></a>
# 10. Hiperparámetros

Cada algoritmo tiene sus propios hiperparámetros. Aquí los más importantes:

## PPO

| Parámetro | Default | Descripción |
|-----------|---------|-------------|
| `learning_rate` | 3e-4 | Tasa de aprendizaje |
| `n_steps` | 2048 | Pasos por actualización |
| `batch_size` | 64 | Tamaño del minibatch |
| `n_epochs` | 10 | Épocas por actualización |
| `gamma` | 0.99 | Factor de descuento |
| `clip_range` | 0.2 | Rango de clipping de PPO |
| `ent_coef` | 0.0 | Coeficiente de entropía |

## DQN

| Parámetro | Default | Descripción |
|-----------|---------|-------------|
| `learning_rate` | 1e-4 | Tasa de aprendizaje |
| `buffer_size` | 1M | Tamaño del replay buffer |
| `batch_size` | 32 | Tamaño del batch |
| `gamma` | 0.99 | Factor de descuento |
| `exploration_fraction` | 0.1 | Fracción de exploración |
| `target_update_interval` | 10000 | Actualización de target network |

In [ ]:
# EJEMPLO: PPO CON HIPERPARÁMETROS CUSTOM
# =======================================

if SB3_AVAILABLE:
    model = PPO(
        "MlpPolicy",
        "CartPole-v1",
        
        # Hiperparámetros de aprendizaje
        learning_rate=1e-3,      # Más alto = aprende más rápido pero menos estable
        gamma=0.99,              # Factor de descuento
        
        # Hiperparámetros de muestreo
        n_steps=1024,            # Pasos antes de cada actualización
        batch_size=64,           # Tamaño del minibatch
        n_epochs=10,             # Épocas de optimización
        
        # Hiperparámetros de PPO específicos
        clip_range=0.2,          # Rango de clipping
        ent_coef=0.01,           # Bonus por entropía (promueve exploración)
        
        # Otros
        verbose=1
    )
    
    model.learn(total_timesteps=20_000)
    
    mean_r, _ = evaluate_policy(model, gym.make("CartPole-v1"), n_eval_episodes=10)
    print(f"\n🎯 Resultado: {mean_r:.1f}")

---
<a id='11-proyectos'></a>
# 11. Proyectos Prácticos

Ahora conectamos con los proyectos en `04_proyectos_avanzados/`:

| Proyecto | Archivo | Algoritmo | Entorno |
|----------|---------|-----------|--------|
| LunarLander | `lunarlander/lunarlander_sb3.py` | PPO, DQN, A2C | LunarLander-v2 |
| Highway | `highway/highway_conduccion.py` | DQN, PPO | highway-env |
| MiniGrid | `minigrid/minigrid_navegacion.py` | PPO | MiniGrid |
| PyBullet | `pybullet/pybullet_robotica.py` | SAC, TD3 | PyBullet |

In [ ]:
# EJEMPLO: LUNARLANDER
# ====================
# Este es uno de los entornos más populares para aprender RL

if SB3_AVAILABLE:
    try:
        # Crear entorno (requiere gymnasium[box2d])
        env = gym.make("LunarLander-v2")
        
        print("🚀 LunarLander-v2")
        print(f"   Observaciones: {env.observation_space}")
        print(f"   Acciones: {env.action_space}")
        print(f"   Acciones disponibles:")
        print(f"     0: No hacer nada")
        print(f"     1: Motor izquierdo")
        print(f"     2: Motor principal")
        print(f"     3: Motor derecho")
        
        # Entrenar con PPO
        print("\n🏋️ Entrenando PPO (esto toma ~1-2 minutos)...")
        model = PPO("MlpPolicy", env, verbose=1)
        model.learn(total_timesteps=50_000)
        
        # Evaluar
        mean_r, std_r = evaluate_policy(model, env, n_eval_episodes=10)
        print(f"\n🎯 Resultado: {mean_r:.1f} ± {std_r:.1f}")
        print(f"   (>200 = aterrizaje exitoso)")
        
        env.close()
        
    except Exception as e:
        print(f"⚠️ Error: {e}")
        print("   Instala Box2D: pip install gymnasium[box2d]")

In [ ]:
# EJEMPLO: ACCIONES CONTINUAS CON SAC
# ====================================

if SB3_AVAILABLE:
    try:
        # Pendulum tiene acciones continuas
        env = gym.make("Pendulum-v1")
        
        print("🎯 Pendulum-v1")
        print(f"   Observaciones: {env.observation_space}")
        print(f"   Acciones: {env.action_space}")
        print(f"   Acción: torque continuo en [-2, 2]")
        
        # SAC es ideal para acciones continuas
        print("\n🏋️ Entrenando SAC...")
        model = SAC("MlpPolicy", env, verbose=1)
        model.learn(total_timesteps=10_000)
        
        # Evaluar
        mean_r, std_r = evaluate_policy(model, env, n_eval_episodes=10)
        print(f"\n🎯 Resultado: {mean_r:.1f} ± {std_r:.1f}")
        print(f"   (Máximo teórico: 0, más negativo = peor)")
        
        env.close()
        
    except Exception as e:
        print(f"⚠️ Error: {e}")

---
# Resumen

## Lo que Aprendimos

| Tema | Concepto Clave |
|------|----------------|
| **SB3 Básico** | `model = PPO(...); model.learn(); model.predict()` |
| **Algoritmos** | PPO (general), DQN (discreto), SAC (continuo) |
| **Políticas** | `MlpPolicy` (vectores), `CnnPolicy` (imágenes) |
| **Paralelo** | `make_vec_env(env_id, n_envs=N)` |
| **Callbacks** | `EvalCallback`, `CheckpointCallback` |
| **Guardar** | `model.save()`, `Model.load()` |

## Próximos Pasos

1. **Explorar los proyectos** en `04_proyectos_avanzados/`
2. **Experimentar con hiperparámetros** en diferentes entornos
3. **Crear tu propio entorno** y entrenarlo con SB3
4. **Leer la documentación**: https://stable-baselines3.readthedocs.io/

---

## Recursos

- **Documentación SB3**: https://stable-baselines3.readthedocs.io/
- **RL Baselines Zoo**: https://github.com/DLR-RM/rl-baselines3-zoo (hiperparámetros optimizados)
- **Hugging Face Hub**: https://huggingface.co/models?library=stable-baselines3 (modelos pre-entrenados)